In [1]:
import requests
import pandas as pd
from datetime import datetime, timedelta, timezone


# Step 1: Fetch reviews from App Store

app_ids = ["1212752482", "1450178837", "1404871703"]   
country = "in"

# Cutoff date = last 180 days
cutoff_date = datetime.now(timezone.utc) - timedelta(days=180)

all_reviews = []

for app_id in app_ids:
    # Get reviews RSS JSON
    r_rev = requests.get(f"https://itunes.apple.com/{country}/rss/customerreviews/id={app_id}/json")
    rev_json = r_rev.json()
    entries = rev_json.get("feed", {}).get("entry", [])

    for e in entries:
        if 'author' in e:
            date_str = e.get("updated", {}).get("label")
            try:
                review_date = datetime.fromisoformat(date_str)
                if review_date.tzinfo is not None:
                    review_date = review_date.astimezone(timezone.utc)
                else:
                    review_date = review_date.replace(tzinfo=timezone.utc)
            except Exception:
                continue

            if review_date >= cutoff_date:
                all_reviews.append({
                    "app_id": app_id,
                    "id": e.get("id", {}).get("label"),
                    "title": e.get("title", {}).get("label"),
                    "rating": e.get("im:rating", {}).get("label"),
                    "version": e.get("im:version", {}).get("label"),
                    "date": review_date.strftime("%Y-%m-%d %H:%M:%S"),
                    "content": e.get("content", {}).get("label")
                })


# Step 2: Convert to DataFrame

reviews_df = pd.DataFrame(all_reviews)

# Convert rating to numeric
reviews_df['rating'] = pd.to_numeric(reviews_df['rating'], errors='coerce')

# Convert date column to datetime.date
reviews_df['date'] = pd.to_datetime(reviews_df['date'], errors='coerce').dt.date
reviews_df = reviews_df.dropna(subset=['date'])
reviews_df

,app_id,id,title,rating,version,date,content
0,1212752482,13376317072,ET Money App Has Become Too Focused on Membership,4,5.1.21,2025-11-09,Just a quick feedback on the ET Money app. Now...
1,1212752482,13361642249,Worst app,1,5.1.21,2025-11-05,Auto debit is not happening and no customer se...
2,1212752482,13342631209,Cut the Clutter. Need for improvement,1,5.1.21,2025-11-01,Need to be simplified instead of making it so ...
3,1212752482,13337187314,Becoming increasingly difficult to get basic d...,2,5.1.21,2025-10-31,"Over the years the app has become too big, too..."
4,1212752482,13323916656,App not working!,1,5.1.19,2025-10-28,My app is not opening from last quite somedays...
...,...,...,...,...,...,...,...
145,1404871703,13321348884,Us markets,3,17.95.0,2025-10-27,Grow need to add us markets also in India
146,1404871703,13320991409,Account Opening,1,17.95.0,2025-10-27,Though KYC was digital and quick account activ...
147,1404871703,13320903272,Worst support I ever got,1,17.95.0,2025-10-27,I wanted change my bank account linked to SIP ...
148,1404871703,13320768913,Requires optimisation,2,17.95.0,2025-10-27,App UI on iPad is not user friendly and it is ...
